 <h2> Domain.com.au Web Scraping</h2>
                             <img src="http://www.domain.com.au/Content/Files/misc/domainLogoHead.gif">

IMPORTING LIBRARIES

In [ ]:
# HTML extraction & Parsin
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib
import math
import os
from constants import Brisbane_suburbs,Melbourne_suburbs,Sydney_suburbs,Adelaide_suburbs,Perth_suburbs,price_range_list,headers
from util import progress_bar,start_from_last_run,update_progress
from random import randint
from time import sleep

# Calculate Latitude & Longitude
from geopy.geocoders import Nominatim
# Calculate Distance from Latitude & Longitude
from math import sin, cos, sqrt, atan2, radians
import emailsend



SET GLOBAL PARAMETERS & POPULATE URLs

In [ ]:
# Change the index to change the city
city_list = ['Sydney','Melbourne','Brisbane','Adelaide','Perth']
city = city_list[1]
if city == 'Sydney':
    suburbs_list = Sydney_suburbs
elif city == 'Melbourne':
    suburbs_list = Melbourne_suburbs
elif city == 'Brisbane':
    suburbs_list = Brisbane_suburbs
elif city == 'Adelaide':
    suburbs_list = Adelaide_suburbs
elif city == 'Perth':
    suburbs_list = Perth_suburbs

# Switch to include price range in URL
include_price = True
# switch to determine if this is first run or continuation
first_run = False


print(f"CHOSEN CITY: {city}")
if not os.path.exists(f"../MLrealestate/data/{city}/"):
    os.mkdir(f"../MLrealestate/data/{city}/")


# Populate URLs list
urls = []
for suburb in suburbs_list:
    if include_price:
        for price in price_range_list:
            urls.append(f"https://www.domain.com.au/sold-listings/{suburb}/?price={price}&excludepricewithheld=1&ssubs=0&page=")

    else:
        urls.append(f"https://www.domain.com.au/sold-listings/{suburb}/?excludepricewithheld=1&ssubs=0&page=")

if not first_run:
    urls = start_from_last_run(city=city,urls=urls)
house_list = []
total_houses = 0



HTTP REQUEST TO GAIN DATA

In [3]:
for url in urls:
    progress_bar(urls.index(url), len(urls))
    max_pages = 50
    for x in range(1,max_pages+1):
        if x >= max_pages:
            break
        full_url = url+str(x)
        print(full_url)
        html_text = requests.get(full_url,headers=headers).text
        sleep(randint(1,5))
        soup = BeautifulSoup(html_text,'lxml')
        if x ==1:
            try:
                max_pages = math.ceil(int(soup.find(attrs = {'data-testid':'summary'}).text.split(' ')[0])/20)
                print(str(max_pages) + ' is the number of pages found')
                if max_pages > 50:
                    print(str(max_pages)+ ' is the number of max pages found but we are capping it to 50')
                    max_pages = 50
            except:
                max_pages = 2
        
                
            
        houses = soup.find_all('li',class_= 'css-1qp9106')
      
        
        i = 0
        for house in houses:
           
            # Extract reference id
            link = house.find('a')
            id =link.get('href').split('-')[-1]
    

            # Extract Data Sold & Method of Sale
            date_method = house.find('span', class_='css-1nj9ymt').text
            date_method_split = date_method.split(" ")
            date = f"{date_method_split[-3]} {date_method_split[-2]} {date_method_split[-1]}"
            if "auction" in date_method:
                method = 'auction'
            elif "private treaty" in date_method:
                method = 'private treaty'
            else:
                method = 'Another Method'

            # Extract Address details
            try:
    
                street = house.find('span', class_='css-iqrvhs',attrs={'data-testid': 'address-line1'}).text.replace(',','')
                
            except:
                print('ISSUE FOUND WITH STREET')
                street = 'No Street Found'
            
            try:
                 address = house.find('span', class_='css-iqrvhs',attrs={'data-testid': 'address-line2'}).text
                 address_split = address.split(' ')
                 postcode = address_split[-1]
                 state = address_split[-2]
                 if len(address_split) == 4:
                    suburb = f"{address_split[0]} {address_split[1]}"
                 elif len(address_split) == 5:
                    suburb = f"{address_split[0]} {address_split[1]} {address_split[2]}"
                 else:
                    suburb = address_split[0]
                
            except:
                print('ISSUE FOUND WITH ADDRESS')
            
               
            

            # Extract house details such number or bedrooms,bathrooms, car spots
            price = house.find('p', class_='css-mgq8yx').text.replace(' price from APM PriceFinder','').replace('$','').replace(',','')
            type = house.find('span', class_='css-693528').text.replace('/ Unit / Flat','')

            try:
                features = house.find('div', class_ ='css-18biwo', attrs = {'data-testid':'property-features-wrapper'})
                bedrooms = features.span.span.text[0]
                if bedrooms == '−' or bedrooms is None:
                    bedrooms = 0
                bathrooms = features.span.next_sibling.span.text[0]
                if bathrooms == '−' or bathrooms is None:
                    bathrooms = 1
                cars = features.span.next_sibling.next_sibling.span.text[0]
                if cars == '−':
                    cars = 0
                if len(list(features.children)) == 4:
                    area = features.span.next_sibling.next_sibling.next_sibling.span.text
                else:
                    area = 0
            except:
                bedrooms = 0
                bathrooms = 0
                cars = 0
                area = 0
                print(f"No features found for {street} {address}")

            # Extract Real Estate agent
            agent = house.find('div', class_='css-6yavch',attrs={'data-testid':'listing-card-lazy-image'})
            if agent is None:
                agent = 'UNKNOWN'
            elif agent.img['alt'][0] == 'P':
                agent = 'UNKNOWN'
            else:
                agent = agent.img['alt'].replace('Logo for ','')
            

            # Extract Advertisemen URL
            ad_url= house.find('a').get('href')
    
            
        
            house_details = {
                'Id' : id,
                'Street' : street,
                'Address' : address,
                'Suburb' : suburb,
                'State' : state,
                'Postcode' : postcode,
                'Type' : type,
                'Date': date,
                'Method' : method,
                'Price' : price,
                'Bedrooms': bedrooms,
                'Bathrooms': bathrooms,
                'Cars' : cars,
                'Area' : area,
                'Agent' : agent,
                'Advertisement_URL' : ad_url
            }
            if (bedrooms == 0 and bathrooms == 0 and cars == 0 and area == 0) or street == 'No Street Found':
                print(f"{street} {address} house was NOT added")
            else:
                house_list.append(house_details)
                i = i + 1
        total_houses = total_houses +   len(house_list)
        print('HOUSES ADDED per QUERY: '+str(i)+ ' out of possible '+str(len(houses)))
        update_progress(city=city,url=url)
        
        if len(house_list) >= 1:
            df = pd.DataFrame(house_list)
            df.to_csv(f"../MLrealestate/data/{city}/{city}_raw.csv",mode='a', header = False, index = False)
            print('TOTAL HOUSES ADDED: '+str(total_houses))
            house_list = []



No features found for 6 Holder Place  BERWICK VIC 3806
6 Holder Place  BERWICK VIC 3806 house was NOT added
HOUSES ADDED per QUERY: 19 out of possible 20
TOTAL HOUSES ADDED: 19749
https://www.domain.com.au/sold-listings/berwick-vic-3806/?price=300000-350000&excludepricewithheld=1&ssubs=0&page=10
No features found for 35 Lorikeet Drive  BERWICK VIC 3806
35 Lorikeet Drive  BERWICK VIC 3806 house was NOT added
No features found for 10 Bushlark Court  BERWICK VIC 3806
10 Bushlark Court  BERWICK VIC 3806 house was NOT added
HOUSES ADDED per QUERY: 18 out of possible 20
TOTAL HOUSES ADDED: 19767
https://www.domain.com.au/sold-listings/berwick-vic-3806/?price=300000-350000&excludepricewithheld=1&ssubs=0&page=11
No features found for 7 Balfour Court  BERWICK VIC 3806
7 Balfour Court  BERWICK VIC 3806 house was NOT added
No features found for 32 Sheldon Drive  BERWICK VIC 3806
32 Sheldon Drive  BERWICK VIC 3806 house was NOT added
No features found for 4 Coniston Avenue  BERWICK VIC 3806
4 Coni

SAVING PANDAS DF TO CSV. PLEASE NOTE: WE ARE APPENDING TO CSV

In [ ]:
df = pd.DataFrame(house_list)
df.to_csv(f"../MLrealestate/data/{city}/{city}_raw1.csv",mode='a', header = False, index = False)
df.tail(6)

Creating Latitude & Longitude columns from address

In [ ]:

df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_raw.csv")

def get_coordinates(street, suburb):
    loc = Nominatim(user_agent="GetLoc")

    try:
        if {street} == 'No Street Found':
            getLoc = loc.geocode(f"{suburb},Australia")
        else:
            getLoc = loc.geocode(f"{street},{suburb},Australia")
        lat = getLoc.latitude
        long = getLoc.longitude
    except:
        try:
            getLoc = loc.geocode(f"{suburb},Australia")
            lat = getLoc.latitude
            long = getLoc.longitude
        except:
            print(f"{street},{suburb},Australia was not able to find lat & long")
            lat = -37.7618477
            long = 144.9805318
    return lat,long


vect_get_coordinates = np.vectorize(get_coordinates)

completed = 0

for i in range(0,len(df.index),1000):
    df_seg = df[i:i+1000]
    df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)
    df_seg.to_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv",mode='a',header = False,index=False)
    completed = completed + len(df_seg)
    print(f"PROCESSED: {completed} RECORDS OUT OF TOTAL {len(df.index)}")
    print(f"PROGRESS: {completed/len(df.index)} % COMPLETED")

    

In [ ]:
df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv")

len(df[df['Latitude'] == -37.7618477])

In [ ]:
def get_coordinates(street, suburb):
    loc = Nominatim(user_agent="GetLoc")

    try:
        if {street} == 'No Street Found':
            getLoc = loc.geocode(f"{suburb},Australia")
        else:
            getLoc = loc.geocode(f"{street},{suburb},Australia")
        lat = getLoc.latitude
        long = getLoc.longitude
    except:
        try:
            getLoc = loc.geocode(f"{suburb},Australia")
            lat = getLoc.latitude
            long = getLoc.longitude
        except:
            print(f"{street},{suburb},Australia was not able to find lat & long")
            lat = -37.7618477
            long = 144.9805318
    return lat,long

df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv")

def fix_lat_lon(row):
    if row['Latitude'] == -37.7618477:
       try:
            row['Latitude'],row['Longitude'] = get_coordinates(row['Street'], row['Suburb'])
           
            print(f"fixed {row['Street']} new lat = {row['Latitude']}  new long= {row['Longitude']}")
       except:
            print(f"Unable to fix {row['Street']}")
    return row


df = df.apply(fix_lat_lon, axis=1)
df.head()      




In [ ]:
df.to_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv", header = True, index = False)

CALCULATE DISTANCE FROM CITY by vectorization function and using Latitude and longitude

In [ ]:


df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv")
la2,lo2 = get_coordinates('No Street Found', city)

def calc_distance(la1,lo1):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(la1)
    lon1 = radians(lo1)
    lat2 = radians(la2)
    lon2 = radians(lo2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
   

vec_cal_distance = np.vectorize(calc_distance)
df['Distance']  = vec_cal_distance(df.Latitude, df.Longitude)
df.to_csv(f"../MLrealestate/data/{city}/{city}_distance.csv",header = True,index=False)
df.head(20)

Clean up Area by removing commas,m2, and converting Ha to m2

In [ ]:
df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_distance.csv")

def convert_area(area):
    if ',' in area:
        area = area.replace(',','')
    if 'm²' in area:
        area = area.replace('m²','')
    if 'ha' in area:
        area = 10000*float(area.replace('ha',''))
    return float(area)
   

vec_convert_area = np.vectorize(convert_area)
df['Area']  = vec_convert_area(df.Area)


df.to_csv(f"../MLrealestate/data/{city}/{city}_area.csv",header = True,index=False)

In [ ]:
import plotly.express as px
df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_distance.csv")
fig = px.scatter_mapbox(df,lat = df['Latitude'],lon = df['Longitude'],zoom =8,color = df['Price'],size = df['Price'], text =df.Street+ df.Address, width = 1100, height = 800, title = city + ' House Prices Map')
fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(margin = {'r':0 , 't': 50, 'l':0 , 'b':10})
fig.show()

Send Email of the collected & transformed data

In [ ]:

emailsend.run(f"../MLrealestate/data/{city}/{city}_area.csv")

TEST

In [ ]:
htmltest = '<li class="is-first-in-list css-1qp9106" data-testid="listing-2018188536"><div class="css-rxp4mi" data-testid="listing-card-wrapper-standardpp"><div class="css-1h8fpgv"><div class="css-1jlleu4" data-testid="listing-card-carousel"></div><div class="css-o1tuxx" data-testid="listing-card-branding"><div class="css-6yavch" data-testid="listing-card-lazy-image"><img alt="Logo for Ray White Thomastown" loading="lazy" src="https://rimh2.domainstatic.com.au/NI3obEa5Dmg64Y2NUggyfdvQoHQ=/170x60/filters:format(jpeg):quality(80)/https://images.domain.com.au/img/Agencys/27311/logo_27311.jpg?buster=2022-12-11"/></div></div><div class="css-tmtv67" data-testid="listing-card-tag"><span class="css-1nj9ymt">Sold at auction 10 Dec 2022</span></div></div><div class="css-1gkcyyc"><div class="css-qrqvvg"><div class="css-9hd67m" data-testid="listing-card-price-wrapper"><p class="css-mgq8yx" data-testid="listing-card-price">$798,000<!-- --> </p><button aria-label="Add 38 Franklin Road to shortlist" class="css-bhcn0k" data-testid="listing-card-shortlist" type="button"><svg aria-hidden="true" class="domain-icon css-jeyium" viewbox="0 0 24 24"><path d="M11.4 17.2l-5.6 3.7a.5.5 0 0 1-.7-.7l2.8-5.5a1 1 0 0 0-.3-1.2L3.2 10a.5.5 0 0 1 .3-1h5.1a1 1 0 0 0 .9-.6l1.9-5a.6.6 0 0 1 1.1 0l1.9 5a1 1 0 0 0 .9.6h5.1a.5.5 0 0 1 .3 1l-4.4 3.5a1 1 0 0 0-.3 1.2l2.8 5.5a.5.5 0 0 1-.8.7l-5.6-3.7a1 1 0 0 0-1 0z" fill="inherit" stroke="currentColor" stroke-miterlimit="10" stroke-width="2"></path></svg></button></div><a class="address is-two-lines css-1y2bib4" href="https://www.domain.com.au/38-franklin-road-lalor-vic-3075-2018188536" rel="noopener"><h2 class="css-bqbbuf" data-testid="address-wrapper"><span class="css-iqrvhs" data-testid="address-line1">38 Franklin Road<!-- -->, </span><span class="css-iqrvhs" data-testid="address-line2"><span>LALOR</span> <span>VIC</span> <span>3075</span></span></h2></a><div class="css-1t41ar7" data-testid="listing-card-features-wrapper"><div class="css-k1qq7e" data-testid="property-features"><div class="css-18biwo" data-testid="property-features-wrapper"><span class="css-1ie6g1l" data-testid="property-features-feature"><span class="css-lvv8is" data-testid="property-features-text-container">3<!-- --> <span class="css-9fxapx" data-testid="property-features-text">Beds</span></span><svg aria-hidden="true" class="domain-icon property-feature__icon css-kry84i" height="24" viewbox="0 0 18 18" width="24"><path d="M3.00002 15.00001v1M15.00002 15.00001v1M6 9h6a4 4 0 0 1 4 4v2H2v-2a4 4 0 0 1 4-4z" fill="none" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2"></path><path d="M5.5 9V7.5a1 1 0 0 1 1-1h1a1 1 0 0 1 1 1V8M9.5 8v-.5a1 1 0 0 1 1-1h1a1 1 0 0 1 1 1V9" fill="none" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round"></path><path d="M15 10V4a1 1 0 0 0-1-1H4a1 1 0 0 0-1 1v6" fill="none" stroke="currentColor" stroke-linejoin="round" stroke-width="2"></path><path d="M1.5 12.5h15" fill="none" stroke="currentColor" stroke-linejoin="round"></path><path d="M0 0h18v18H0z" fill="none"></path></svg></span><span class="css-1ie6g1l" data-testid="property-features-feature"><span class="css-lvv8is" data-testid="property-features-text-container">2<!-- --> <span class="css-9fxapx" data-testid="property-features-text">Baths</span></span><svg aria-hidden="true" class="domain-icon property-feature__icon css-kry84i" height="24" viewbox="0 0 18 18" width="24"><path d="M5 15v1M13 15v1" fill="none" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2"></path><circle cx="13.5" cy="6.5" fill="currentColor" r=".5"></circle><circle cx="11.5" cy="4.5" fill="currentColor" r=".5"></circle><circle cx="11.5" cy="6.5" fill="currentColor" r=".5"></circle><circle cx="9.5" cy="6.5" fill="currentColor" r=".5"></circle><path d="M3 9h12v2a4 4 0 0 1-4 4H7a4 4 0 0 1-4-4V9zM3 9V5a3 3 0 0 1 3-3" fill="none" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2"></path><path d="M6 4.3V2a1 1 0 0 1 1-1h2.3a.7.7 0 0 1 .5 1.2L7.2 4.8A.7.7 0 0 1 6 4.3zM2 9h14" fill="none" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2"></path><path d="M0 0h18v18H0z" fill="none"></path></svg></span><span class="css-1ie6g1l" data-testid="property-features-feature"><span class="css-lvv8is" data-testid="property-features-text-container">6<!-- --> <span class="css-9fxapx" data-testid="property-features-text">Parking</span></span><svg aria-hidden="true" class="domain-icon property-feature__icon css-kry84i" height="24" viewbox="0 0 18 18" width="24"><path d="M16.5 6.5h-2M3.5 6.5h-2M7.5 11.5h3M3.5 7.5h11" fill="none" stroke="currentColor" stroke-linecap="round" stroke-miterlimit="10"></path><path d="M15.7 8.8l-1.6-2.6-.2-.4-.5-1.4A2 2 0 0 0 11.6 3H6.4a2 2 0 0 0-1.9 1.4l-.4 1.4-.2.4-1.6 2.6a2 2 0 0 0-.3 1V13a2 2 0 0 0 2 2h10a2 2 0 0 0 2-2V9.9a2 2 0 0 0-.3-1.1z" fill="none" stroke="currentColor" stroke-linejoin="round" stroke-width="2"></path><path d="M1.5 10.5H5a.5.5 0 0 1 .5.5v.5a1 1 0 0 1-1 1h-3v-2zM13 10.5h3.5v2h-3a1 1 0 0 1-1-1V11a.5.5 0 0 1 .5-.5z" fill="none" stroke="currentColor" stroke-linejoin="round"></path><path d="M3 14h3v2a1 1 0 0 1-1 1H4a1 1 0 0 1-1-1v-2zM12 14h3v2a1 1 0 0 1-1 1h-1a1 1 0 0 1-1-1v-2z" fill="currentColor"></path></svg></span><span class="css-1ie6g1l" data-testid="property-features-feature"><span class="css-lvv8is" data-testid="property-features-text-container">612m²<!-- --> </span><svg aria-hidden="true" class="domain-icon property-feature__icon css-kry84i" height="24" viewbox="0 0 18 18" width="24"><line fill="none" stroke="currentColor" x1="2.5" x2="2.5" y1="3" y2="11"></line><path d="M1.3,3H3.7c.2,0,.4-.2.2-.4L2.7,1.1a.3.3,0,0,0-.5,0L1.1,2.6C.9,2.8,1.1,3,1.3,3Z" fill="currentColor"></path><path d="M3.7,11H1.3c-.2,0-.4.2-.2.4l1.2,1.5a.3.3,0,0,0,.5,0l1.2-1.5C4.1,11.2,3.9,11,3.7,11Z" fill="currentColor"></path><line fill="none" stroke="currentColor" x1="16" x2="7" y1="15.5" y2="15.5"></line><path d="M15,14.3v2.4c0,.2.2.4.4.2l1.5-1.2a.3.3,0,0,0,0-.5l-1.5-1.2C15.2,13.9,15,14.1,15,14.3Z" fill="currentColor"></path><path d="M7,16.7V14.3c0-.2-.2-.4-.4-.2L5.1,15.3a.3.3,0,0,0,0,.5l1.5,1.2C6.8,17.1,7,16.9,7,16.7Z" fill="currentColor"></path><rect fill="none" height="10" stroke="currentColor" stroke-width="2" width="10" x="6" y="2"></rect><rect fill="none" height="18" width="18"></rect></svg></span></div></div><div class="css-11n8uyu"><span class="css-693528">House</span></div></div></div></div></div></li>'